In [0]:
!pip install geffnet
!pip install timm

     |████████████████████████████████| 163kB 4.0MB/s 


In [0]:
import geffnet
from torchsummary import summary
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import sys
from timm.optim.radam import RAdam

In [0]:
torch.manual_seed(0)
use_GPU = torch.cuda.is_available()
device = torch.device("cuda" if use_GPU else "cpu")
if use_GPU:
    torch.cuda.manual_seed(0)
print("Using GPU: {}".format(use_GPU))

Using GPU: True


In [0]:
def ImageProcessing():
    transform = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224),
                                    transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

    train_dat = datasets.CIFAR10(root=sys.path[0] + "/data/CIFAR10", train=True, download=True, transform=transform)

    train_loader = DataLoader(train_dat, batch_size=32, shuffle=False, num_workers=2)

    test_dat = datasets.CIFAR10(root=sys.path[0] + '/data/CIFAR10', train=False, download=True, transform=transform)

    test_loader = torch.utils.data.DataLoader(test_dat, batch_size=1, shuffle=False, num_workers=2)

    return train_loader, test_loader

In [0]:
def training(model, train_loader, optimizer, criterion, epoch):
    training_loss = 0
    model.train()
    bi = 200

    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, targets)

        loss.backward()

        optimizer.step()

        training_loss += loss.item()

        if batch_idx % bi == bi-1:  # print every 2000 mini-batches
            print('[epoch: %d, batch: %5d] loss: %.4f' % (epoch + 1, batch_idx + 1, training_loss / bi))
            training_loss = 0.0


In [0]:
def testing(model, test_loader, criterion):
    test_loss = 0.0
    correct = 0
    total = 0
    model.eval()

    with torch.no_grad():

        for inputs, targets in test_loader:
          
            inputs = inputs.to(device)

            targets = targets.to(device)

            outputs = model(inputs)

            test_loss += criterion(outputs, targets).item()

            _, predicted = torch.max(outputs.data, 1)

            correct += (predicted == targets).sum().item()

            total += 1

    print("\nTest set: Average loss: {:.4f}, Accuracy: {:.4f}\n".format(test_loss / total,
                                                                        correct / len(test_loader.dataset)))
    return test_loss / total, correct / len(test_loader.dataset)

In [0]:
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [0]:
def fine_tuning(model):

    epochs = 20
    lr = 0.0001

    test_acc = []
    test_loss = []

    model.to(device)

    summary(model,(3,224,224),batch_size=10)

    # optimizer = torch.optim.RMSprop(params_to_update, lr = 0.0001, weight_decay =1e-5)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.99, weight_decay=1e-5)
    # optimizer = torch.optim.SGD(params_to_update, lr=config["lr"], momentum=config["momentum"])

    # train_loader, test_loader = ImageProcessingForFolder("****************", 32, resolution, img_stats)
    
    # optimizer = torch.optim.Adam(model.parameters(),lr=lr,  eps=1e-5, weight_decay=1e-5)
    
    # reduce_lr = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.6)
    reduce_lr = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=1)

    train_loader, test_loader = ImageProcessing()

    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        
        print_learning_rate(optimizer,epoch+1)

        training(model, train_loader, optimizer, criterion, epoch)

        loss, acc = testing(model, test_loader, criterion)

        test_loss.append(loss)

        test_acc.append(acc)

        # adjust_learning_rate(0.0001, optimizer,epoch+1)

        reduce_lr.step(loss)


    fig, (ax1,ax2) = plt.subplots(1,2)
    ax1.plot(list(range(1,epochs+1)), test_acc)
    ax1.set(xlabel='epochs', ylabel='test accuracy')
    ax2.plot(list(range(1,epochs+1)), test_loss)
    ax2.set(xlabel='epochs', ylabel='test loss')
    fig.tight_layout(pad=4.0)
    ax2.set_xticks(np.arange(1, epochs+1, step=1))
    ax1.set_xticks(np.arange(1, epochs+1, step=1))

    plt.show()
    
    return model


In [0]:
def adjust_learning_rate(lr, optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 0.97 every 2.4 epochs"""
    lr = lr * (0.9 ** (epoch // 2))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def print_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    for param_group in optimizer.param_groups:
        print("Epoch: [{}] Current learning rate (lr) = {}".format(
                                                    epoch, param_group['lr']))

In [0]:
def main(md):

    classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    # classes = (
    # 'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    # 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    # 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    # 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    # 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    # 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    # 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    # 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    # 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    # 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    # 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    # 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    # 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    # 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    # 'worm')

    train_loader, test_loader = ImageProcessing()
    
    # print(len(classes))
    # imageshow(train_loader, classes)

    momentum = 0.9
    epochs = 20
    decay = 1e-5
    lr = 0.001

    model = md.to(device)

    summary(model,(3,224,224),batch_size=10)

    criterion = nn.CrossEntropyLoss()
    
    # optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=decay)
    # optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer = RAdam(model.parameters(),lr=lr,  eps=1e-5, weight_decay=decay)

    # reduce_lr = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
    reduce_lr = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=1)

    for epoch in range(epochs):
        print_learning_rate(optimizer,epoch+1)
        training(model, train_loader, optimizer, criterion, epoch)
        loss,_ = testing(model, test_loader, criterion)
        # adjust_learning_rate(lr, optimizer,epoch+1)
        reduce_lr.step(loss)

    PATH = './cifar_net.pth'
    torch.save(model.state_dict(), PATH)


In [0]:
print(torch.hub.list('rwightman/gen-efficientnet-pytorch'))

md = geffnet.create_model('efficientnet_b0',pretrained=False)

md.classifier = torch.nn.Linear(1280,10,bias=True)

main(md)

Downloading: "https://github.com/rwightman/gen-efficientnet-pytorch/archive/master.zip" to /root/.cache/torch/hub/master.zip


['efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_es', 'fbnetc_100', 'mixnet_l', 'mixnet_m', 'mixnet_s', 'mixnet_xl', 'mnasnet_a1', 'mnasnet_b1', 'mobilenetv3_large_100', 'mobilenetv3_rw', 'spnasnet_100', 'tf_efficientnet_b0', 'tf_efficientnet_b0_ap', 'tf_efficientnet_b0_ns', 'tf_efficientnet_b1', 'tf_efficientnet_b1_ap', 'tf_efficientnet_b1_ns', 'tf_efficientnet_b2', 'tf_efficientnet_b2_ap', 'tf_efficientnet_b2_ns', 'tf_efficientnet_b3', 'tf_efficientnet_b3_ap', 'tf_efficientnet_b3_ns', 'tf_efficientnet_b4', 'tf_efficientnet_b4_ap', 'tf_efficientnet_b4_ns', 'tf_efficientnet_b5', 'tf_efficientnet_b5_ap', 'tf_efficientnet_b5_ns', 'tf_efficientnet_b6', 'tf_efficientnet_b6_ap', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b7', 'tf_efficientnet_b7_ap', 'tf_efficientnet_b7_ns', 'tf_efficientnet_b8', 'tf_efficientnet_b8_ap', 'tf_efficientnet_cc_b0_4e', 'tf_efficientnet_cc_b0_8e', 'tf_efficientnet_cc_b1_8e', 'tf_efficientnet_el', 'tf_efficientnet_em', 'tf

Extracting /data/CIFAR10/cifar-10-python.tar.gz to /data/CIFAR10
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [10, 32, 112, 112]             864
       BatchNorm2d-2         [10, 32, 112, 112]              64
          SwishJit-3         [10, 32, 112, 112]               0
            Conv2d-4         [10, 32, 112, 112]             288
       BatchNorm2d-5         [10, 32, 112, 112]              64
          SwishJit-6         [10, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [10, 32, 1, 1]               0
            Conv2d-8              [10, 8, 1, 1]             264
          SwishJit-9              [10, 8, 1, 1]               0
           Conv2d-10             [10, 32, 1, 1]             288
    SqueezeExcite-11         [10, 32, 112, 112]               0
           Conv2d-12         [10, 16, 112, 112]             512
